# Identify Phone Numbers Containing Milford Area Code

In [27]:
import requests
import pandas as pd

#can replace files -- require one with area codes and other with phone numbers to be iterated
milford = pd.read_csv('Input Data Files/Milford Area Codes.csv')
unknown = pd.read_csv('Input Data Files/Unknown Phone Numbers.csv')

#casts phone numbers to strings so they can be parsed and compared properly
milford['joined'] = milford['NH area code'].astype(str) + milford['milford prefix'].astype(str)
unknown['phone_number'] = unknown['phone_number'].astype(str)

#creates list of Milford numbers
milford_list = list(milford['joined'])
unknown['is_milford'] = unknown['phone_number'].apply(lambda x: True if x[:6] in milford_list else False)
milford_numbers = list(unknown[unknown['is_milford']].phone_number)

In [26]:
print(milford_numbers)

['6036724016', '6033162901', '6033167714', '6037211646', '6036738057', '6036736262', '6032496478', '6038012034', '6035540183', '6038015408', '6037219340', '6037214276', '6037217334', '6036732086', '6035548477', '6033166600', '6038012018', '6032494631', '6036730313', '6033161918', '6035545817', '6038018735', '6036723207', '6038012740', '6036722694', '6033167090', '6032492606', '6035542531', '6036720829', '6036737342', '6036726522', '6036733862', '6037213749', '6033161923', '6037216077', '6037210426', '6035545454', '6032492725', '6033167964', '6037216917', '6036727131', '6032496709', '6035546924', '6036725483', '6033163952', '6038013395', '6035542043', '6038013139', '6036734703', '6036725418', '6035545251', '6032497800', '6033160027', '6035546412', '6036734205', '6032498141', '6033161956', '6036721890', '6038011091', '6032491324', '6038015667', '6036722556', '6033166513', '6032493770', '6037219645', '6033163970', '6033168803', '6036739903', '6036732218', '6032496335', '6032497710', '6038

# Iterate Through Milford Phone Numbers and Retrieve Output from Ekata 

### Option 1 : Create Separate / Unique CSV File of Output -- Stores Output in New Dataframe

In [28]:
import requests
import pandas as pd

#creates new dataframe to store data
phone_numbers = pd.DataFrame(columns=['Phone_Number', 'Is_Valid', 'Line_Type', 'Carrier', 'Is_Prepaid', 'Is_Commercial', 'Name', 'Firstname', 'Lastname', 'Age_Range', 'Gender', 'City', 'Postal_Code', 'Is_Active'])

#replace api_key with active Ekata 'Reverse Phone' trial / account
for num in milford_numbers:
    #r = requests.get('https://api.ekata.com/3.1/phone?api_key=96cb6784a88c4f65b58c8044086bf000&phone={phone}'.format(phone = num))
    d = dict(r.json())
    
    number = d['phone_number']
    is_valid = d['is_valid']
    line_type = d['line_type']
    carrier = d['carrier']
    is_prepaid = d['is_prepaid']
    is_commercial = d['is_commercial'] 
    try:
        name = d['belongs_to']['name']
        firstname = d['belongs_to']['firstname']
        lastname = d['belongs_to']['lastname']
        age_range = d['belongs_to']['age_range']
        gender = d['belongs_to']['gender']
    except: 
        name = None
        firstname = None
        lastname = None
        age_range = None
        gender = None
        
    try:
        city = d['current_addresses'][0]['city'] 
        postal_code = d['current_addresses'][0]['postal_code'] 
        is_active = d['current_addresses'][0]['is_active'] 
    except:
        city = None
        postal_code = None
        is_active = None
    
    #adds output to new dataframe
    entry = {'Phone_Number': number,'Is_Valid':is_valid, 'Line_Type':line_type, 'Carrier': carrier, 'Is_Prepaid':is_prepaid, 'Is_Commercial':is_commercial, 'Name': name, 'Firstname':firstname, 'Lastname':lastname, 'Age_Range':age_range, 'Gender':gender, 'City':city, 'Postal_Code':postal_code, 'Is_Active':is_active}
    phone_numbers = phone_numbers.append(entry, ignore_index = True)


KeyError: 'phone_number'

In [29]:
phone_numbers

,Phone_Number,Is_Valid,Line_Type,Carrier,Is_Prepaid,Is_Commercial,Name,Firstname,Lastname,Age_Range,Gender,City,Postal_Code,Is_Active


In [ ]:
phone_numbers.to_csv('Milford Phone Numbers.csv')

### Option 2 : Create Master File from Which Output is Read and Written into Directly

##### 1 : Create Master File -- Append Existing Output and Flag Unsearched Numbers

In [38]:
import requests
import pandas as pd

#reads data of current Milford numbers and other numbers to be searched
bq = pd.read_csv('Input Data Files/Milford_Big Query.csv')
ran = pd.read_csv("Milford Numbers.csv")

#creates master file dataframe
master = pd.DataFrame(columns = ['Phone_Number', 'Checked', 'Is_Valid', 'Line_Type', 'Carrier', 'Is_Prepaid', 'Is_Commercial', 'Name', 'Firstname', 'Lastname', 'Age_Range', 'Gender', 'City', 'Postal_Code', 'Is_Active'])

#formats files so they can be indexed properly
#1. deletes index column, 2. casts phone numbers to strings, 3. formats phone number to match Ekata output
ran = ran.drop(ran.columns[0], axis = 1)
bq['phone_number'] = "+1" + bq['phone_number'].astype(str)
ran['Phone_Number'] = "+" + ran['Phone_Number'].astype(str)

checked_nums = list(ran['Phone_Number'])

#iterates through all numbers
for num in bq['phone_number']:
    
    #Case 1 : Number has already been searched -- retrieve output and append to master file
    if num in checked_nums: 
        checked = True
        
        index = checked_nums.index(num)
        data = ran.loc[index]
        
        phone = data['Phone_Number']
        valid = data['Is_Valid']
        line = data['Line_Type']
        carrier = data['Carrier']
        prepaid = data['Is_Prepaid']
        commercial = data['Is_Commercial']
        name = data['Name']
        firstname = data['Firstname']
        lastname = data['Lastname']
        age = data['Age_Range']
        gender = data['Gender']
        city = data['City']
        postal = data['Postal_Code']
        active = data['Is_Active']
        
        entry = {'Phone_Number': phone, 'Checked':checked, 'Is_Valid':valid, 'Line_Type':line, 'Carrier': carrier, 'Is_Prepaid':prepaid, 'Is_Commercial':commercial, 'Name': name, 'Firstname':firstname, 'Lastname':lastname, 'Age_Range':age, 'Gender':gender, 'City':city, 'Postal_Code':postal, 'Is_Active':active}
    
    #Case 2 : Number has NOT been searched -- append number to master file and flag it
    else:
        checked = False
        entry = {'Phone_Number':num, 'Checked':checked}
    
    master = master.append(entry, ignore_index = True)

In [ ]:
master.to_csv('Master File.csv')

##### 2 : Run Unsearched Numbers -- Read From and Append to Master File

###### !! Can Be Run in Isolation !!

In [36]:
import pandas as pd
import requests

master = pd.read_csv('Master File.csv')

#formats file so it can be read properly
#1. deletes index column, 2. casts phone numbers to strings, 3. formats phone number to match Ekata output
master = master.drop(master.columns[0], axis = 1)
master['Phone_Number'] = "+" + master['Phone_Number'].astype(str)


#iterates through all numbers (via indices)
for index in range(len(master)):
    
    num = master['Phone_Number'][index][1::]
    checked = master['Checked'][index]

    #replace api_key with active Ekata 'Reverse Phone' trial / account
    #retrieves Ekata output and directly updates the master file
    if checked == False:
        
        #r= requests.get('https://api.ekata.com/3.1/phone?api_key=441b0c9fc01a4cb0ba7b4f790d3e68aa&phone={phone}'.format(phone = num))
        d = dict(r.json())
        
        master.at[index, 'Phone_Number'] = d['phone_number']
        master.at[index, 'Checked'] = True
        master.at[index, 'Is_Valid'] = d['is_valid']
        master.at[index, 'Line_Type'] = d['line_type']
        master.at[index, 'Carrier'] = d['carrier']
        master.at[index, 'Is_Prepaid'] = d['is_prepaid']
        master.at[index, 'Is_Commercial'] = d['is_commercial'] 
        try:
            master.at[index, 'Name'] = d['belongs_to']['name']
            master.at[index, 'Firstname'] = d['belongs_to']['firstname']
            master.at[index, 'Lastname'] = d['belongs_to']['lastname']
            master.at[index, 'Age-Range'] = d['belongs_to']['age_range']
            master.at[index, 'Gender'] = d['belongs_to']['gender']
        except: 
            master.at[index, 'Name'] = None
            master.at[index, 'Firstname'] = None
            master.at[index, 'Lastname'] = None
            master.at[index, 'Age_Range'] = None
            master.at[index, 'Gender'] = None

        try:
            master.at[index, 'City'] = d['current_addresses'][0]['city'] 
            master.at[index, 'Postal_Code'] = d['current_addresses'][0]['postal_code'] 
            master.at[index, 'Is_Active'] = d['current_addresses'][0]['is_active'] 
        except:
            master.at[index, 'City'] = None
            master.at[index, 'Postal_Code'] = None
            master.at[index, 'Is_Active'] = None

            
#saves master file            
master.to_csv('Master File.csv')

FileNotFoundError: [Errno 2] File b'Master File.csv' does not exist: b'Master File.csv'

In [37]:
master[master['Checked']]

,Phone_Number,Checked,Is_Valid,Line_Type,Carrier,Is_Prepaid,Is_Commercial,Name,Firstname,Lastname,Age_Range,Gender,City,Postal_Code,Is_Active,Age-Range
0,+16036724016,True,True,FixedVOIP,Comcast,False,False,Tempatu Sanford,Tempatu,Sanford,NaN,NaN,Milford,3055.0,NaN,NaN
1,+16033162901,True,True,Mobile,Verizon Wireless,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Milford,3055.0,NaN,NaN
2,+16033167714,True,True,Mobile,Verizon Wireless,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Milford,3055.0,NaN,NaN
3,+16037211646,True,True,Mobile,Verizon Wireless,NaN,False,Gigi M Ciaccia,Gigi,Ciaccia,"{'from': 55, 'to': 59}",NaN,Peterborough,3458.0,True,NaN
4,+16036738057,True,True,Landline,Consolidated Communications,False,NaN,NaN,NaN,NaN,NaN,NaN,Milford,3055.0,NaN,NaN
5,+16036736262,True,True,Landline,Consolidated Communications,False,NaN,NaN,NaN,NaN,NaN,NaN,Milford,3055.0,NaN,NaN
6,+16032496478,True,True,Mobile,Verizon Wireless,NaN,False,John Chiarella,John,Chiarella,NaN,NaN,Milford,3055.0,NaN,NaN
7,+16038012034,True,True,Mobile,Verizon Wireless,NaN,False,John Burton Roo,John,Roo,NaN,NaN,Milford,3055.0,NaN,NaN
8,+16035540183,True,True,Mobile,Verizon Wireless,NaN,False,Jessie R Duquette,Jessie,Duquette,"{'from': 30, 'to': 34}",Female,Milford,3055.0,True,NaN
9,+16038015408,True,True,Mobile,Verizon Wireless,NaN,False,Ms. Kirsten A White,Kirsten,White,"{'from': 30, 'to': 34}",Female,South Grafton,1560.0,NaN,NaN
